# Sensitivity analysis for the DFN

Example showing how to perform sensitivity analysis for the DFN with PyBaMM

In [1]:
%pip install pybamm -q
import pybamm
import numpy as np

Note: you may need to restart the kernel to use updated packages.


Load model

In [14]:
model = pybamm.lithium_ion.SPMe()

Before performing a sensitivity analysis, we scale the parameters with their reference values. Some parameters are functions of states, but we can evaluate them at appropriate values of the states to obtain a reference value. In this notebook, we choose to study the effect on the voltage of:
- negative particle diffusivity (via Ds_n)
- positive particle diffusivity (via Ds_p)
- electrolyte diffusivity (via D_e)
- electrolyte conductivity (via kappa_e)
- negative electrode kinetics (via j0_n)
- positive electrode kinetics (via j0_p)

In [15]:
param = model.default_parameter_values
# Get reference values for evaluating functions
ce_ref = param["Typical electrolyte concentration [mol.m-3]"]
csn_ref = param["Maximum concentration in negative electrode [mol.m-3]"]
csp_ref = param["Maximum concentration in positive electrode [mol.m-3]"]
T_ref = param["Reference temperature [K]"]
# Evaluate functions at reference values
Dsn_ref = param["Negative electrode diffusivity [m2.s-1]"](0.5, T_ref).evaluate()
Dsp_ref = param["Positive electrode diffusivity [m2.s-1]"](0.5, T_ref).evaluate()
De_ref = param["Electrolyte diffusivity [m2.s-1]"](ce_ref, T_ref).evaluate()
kappae_ref = param["Electrolyte conductivity [S.m-1]"](ce_ref, T_ref).evaluate()
j0n_ref = param.evaluate(param["Negative electrode exchange-current density [A.m-2]"](ce_ref, 0.5 * csn_ref, T_ref))
j0p_ref = param.evaluate(param["Positive electrode exchange-current density [A.m-2]"](ce_ref, 0.5 * csp_ref, T_ref))

In [16]:
param["Negative electrode diffusivity [m2.s-1]"] = Dsn_ref * pybamm.InputParameter("Dsn")
param["Positive electrode diffusivity [m2.s-1]"] = Dsp_ref * pybamm.InputParameter("Dsp")
# param["Electrolyte diffusivity [m2.s-1]"] = De_ref * pybamm.InputParameter("D_e")
# param["Electrolyte conductivity [S.m-1]"] = kappae_ref * pybamm.InputParameter("kappa_e")
# param["Negative electrode exchange-current density [A.m-2]"] = j0n_ref * pybamm.InputParameter("j0n")
# param["Positive electrode exchange-current density [A.m-2]"] = j0p_ref * pybamm.InputParameter("j0p")

Create simulation, run and read solution

In [19]:
solver = pybamm.CasadiSolver(mode="fast", sensitivity=True)
sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
solution = sim.solve(t_eval=np.linspace(0,3600), inputs={"Dsn": 1, "Dsp": 1})

In [20]:
solution.solve_time

1.7811190900000042

Since we have not specified the parameter values when solving, the resulting solution contains _symbolic_ variables, such as the voltage

In [17]:
solver = pybamm.CasadiSolver(mode="fast")
sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
solution = sim.solve(t_eval=np.linspace(0,3600), inputs={"Dsn": 1, "Dsp": 1})

In [18]:
solution.solve_time

0.005656635999997661

In [11]:
V = solution["Terminal voltage [V]"]
V

Now we can evaluate the voltage at specific values for the input parameters to get both the value

In [12]:
V.sensitivity

{'all': DM([0, 0.000305216, 0.000323451, 0.000307878, 0.000281315, 0.000252542, 0.000226269, 0.00020523, 0.000191031, 0.000184763, 0.000187613, 0.000201625, 0.000230913, 0.000283916, 0.000377805, 0.000547086, 0.000859691, 0.00144399, 0.00252183, 0.00440237, 0.00728527, 0.0106768, 0.0129031, 0.0123404, 0.00953625, 0.00642767, 0.00416935, 0.00284452, 0.00214823, 0.00179002, 0.00157891, 0.00140318, 0.00120162, 0.000947612, 0.000645371, 0.0003332, 8.76979e-05, 2.15342e-05, 0.000267893, 0.000949333, 0.00213811, 0.00382395, 0.00590563, 0.00821168, 0.0105401, 0.0127, 0.0145411, 0.0159704, 0.0169657, 0.017614]),
 'Dsn': DM([0, 0.000305216, 0.000323451, 0.000307878, 0.000281315, 0.000252542, 0.000226269, 0.00020523, 0.000191031, 0.000184763, 0.000187613, 0.000201625, 0.000230913, 0.000283916, 0.000377805, 0.000547086, 0.000859691, 0.00144399, 0.00252183, 0.00440237, 0.00728527, 0.0106768, 0.0129031, 0.0123404, 0.00953625, 0.00642767, 0.00416935, 0.00284452, 0.00214823, 0.00179002, 0.00157891, 0

In [ ]:
%%time
V.value({"Dsn": 1, "Dsp": 1, "D_e": 1, "kappa_e": 1, "j0n": 1, "j0p": 1})

and sensitivity

In [ ]:
%%time
sens = V.sensitivity({"Dsn": 1, "Dsp": 1, "D_e": 1, "kappa_e": 1, "j0n": 1, "j0p": 1})

In [ ]:
sens

In [13]:
inputs = {k: 1 for k in V.symbolic_inputs_dict.keys()}
h = 1e-6
for k in inputs.keys():
    V_down = V.value(inputs)
    inputs[k] = 1 + h
    V_up = V.value(inputs)
    sens = (V_up - V_down) / h
    print(sens)
    inputs[k] = 1

AttributeError: 'ProcessedVariable' object has no attribute 'symbolic_inputs_dict'

In [ ]:
inputs